In [16]:
%pip install streamlit qdrant-client numpy pandas


  Using cached streamlit-1.42.1-py2.py3-none-any.whl.metadata (8.9 kB)
  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached cachetools-5.5.1-py3-none-any.whl.metadata (5.4 kB)
  Using cached pyarrow-19.0.0-cp310-cp310-win_amd64.whl.metadata (3.4 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached watchdog-6.0.0-py3-none-win_amd64.whl.metadata (44 kB)
  Using cached GitPython-3.1.44-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached narwhals-1.27.1-py3-none-any.whl.metadata (10 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
  Using cached jsonschema_specifications-2024.10.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached referencing-0.36.2-py3-none-any.whl.metadata (2.8 kB)
  Using cached rpds_py-0.22.3-cp310-cp310-wi


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [61]:
%pip install -U qdrant-client

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [88]:
import numpy as np
import pandas as pd
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import Distance
from qdrant_client.http.models import VectorParams

In [82]:
# Sample dataset (subset of your sample data)
data = [
    {"ID": 1001, "Name": "vinodkumar nallamogla", "Age": 34},
    {"ID": 1002, "Name": "vinod katari", "Age": 32},
    {"ID": 1003, "Name": "vinod yerra", "Age": 30},
    {"ID": 1004, "Name": "bharath moti", "Age": 25},
    {"ID": 1005, "Name": "bharath motikayala", "Age": 26},
    {"ID": 1006, "Name": "ramesh katari", "Age": 29},
    {"ID": 1007, "Name": "ramesh dugasani", "Age": 30},
    {"ID": 1008, "Name": "satish dugasasni", "Age": 38},
    {"ID": 1009, "Name": "venu yerra", "Age": 42},
    {"ID": 1010, "Name": "sasi dugasani", "Age": 36},
]

In [44]:
# Create a DataFrame
df = pd.DataFrame(data)

In [45]:
# Number of records (we'll use this for the length of the one-hot vector)
n_records = len(df)

In [46]:
print(n_records)

10


In [47]:
# 1. Generate Sparse Vectors (One-Hot Encoding)
def generate_one_hot(index, length):
    vec = [0.0] * length
    vec[index] = 1.0
    return vec

df['sparse_vector'] = [generate_one_hot(i, n_records) for i in range(n_records)]

In [49]:

# 2. Generate Zero Vectors (All Zeros)
df['zero_vector'] = [[0.0] * n_records for _ in range(n_records)]

In [51]:
#3. Generate Custom Vectors using Uniform Distribution
#    Let's generate a 10-dimensional vector with values from U(0,1)
df['uniform_vector'] = [np.random.uniform(0, 1, 10).round(2).tolist() for _ in range(n_records)]


In [53]:
# 4. Generate Custom Vectors using Normal Distribution
#    For example, with mean=0.5 and std=0.1, 10-dimensional
df['normal_vector'] = [np.random.normal(0.5, 0.1, 10).round(2).tolist() for _ in range(n_records)]

In [55]:
# Display the DataFrame with the generated vectors
print(df[['ID', 'Name', 'Age', 'sparse_vector', 'zero_vector', 'uniform_vector', 'normal_vector']])

     ID                   Name  Age  \
0  1001  vinodkumar nallamogla   34   
1  1002           vinod katari   32   
2  1003            vinod yerra   30   
3  1004           bharath moti   25   
4  1005     bharath motikayala   26   
5  1006          ramesh katari   29   
6  1007        ramesh dugasani   30   
7  1008       satish dugasasni   38   
8  1009             venu yerra   42   
9  1010          sasi dugasani   36   

                                       sparse_vector  \
0  [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1  [0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
2  [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
3  [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
4  [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...   
5  [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...   
6  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...   
7  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...   
8  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...   
9  [0.0, 0

In [56]:
# ---------------------------
# Connect to Qdrant Cloud
# ---------------------------
# Replace these with your actual Qdrant Cloud details:
client = QdrantClient(
    url="https://64c9b924-a747-4c21-874f-f6901dc0431e.us-east-1-0.aws.cloud.qdrant.io:6333",  # e.g., "your-cluster-1234.us-east1.qdrant.io"
    api_key="vgywMs0OrU6zw1BkgnJ6sQ8CAbp2XJ7spYIgQFoRaEjAqCAXLlp3Rg",
    https=True
)


In [98]:
collections = {
    "sparse": "sparse_vectors",
    "zero": "zero_vectors",
    "uniform": "uniform_vectors",
    "normal": "normal_vectors"
}

In [99]:
def create_collection_if_not_exists(collection_name, vector_size):
    if not client.collection_exists(collection_name):
        print(f"Creating collection '{collection_name}'...")
        client.create_collection(
            collection_name=collection_name,
            # vectors_config={
            #     "default": VectorParams(size=vector_size, distance=Distance.COSINE)
            # }
            vectors_config=models.VectorParams(size=vector_size, distance=models.Distance.COSINE)
        )
    else:
        print(f"Collection '{collection_name}' already exists.")

In [100]:
# For sparse and zero vectors, the vector dimension equals n_records; for uniform and normal, dimension is 10.
create_collection_if_not_exists(collections["sparse"], n_records)
create_collection_if_not_exists(collections["zero"], n_records)
create_collection_if_not_exists(collections["uniform"], 10)
create_collection_if_not_exists(collections["normal"], 10)


Collection 'sparse_vectors' already exists.
Creating collection 'zero_vectors'...
Creating collection 'uniform_vectors'...
Creating collection 'normal_vectors'...


In [102]:
from qdrant_client.http.models import PointStruct

# ---------------------------
# Upsert Data into Each Collection
# ---------------------------
points_sparse, points_zero, points_uniform, points_normal = [], [], [], []

for _, row in df.iterrows():
    base_payload = {
        "Name": row["Name"],
        "Age": row["Age"],
    }

    points_sparse.append(PointStruct(id=int(row["ID"]), vector=row["sparse_vector"], payload=base_payload))
    points_zero.append(PointStruct(id=int(row["ID"]), vector=row["zero_vector"], payload=base_payload))
    points_uniform.append(PointStruct(id=int(row["ID"]), vector=row["uniform_vector"], payload=base_payload))
    points_normal.append(PointStruct(id=int(row["ID"]), vector=row["normal_vector"], payload=base_payload))

# Upsert to Qdrant collections
client.upsert(collection_name=collections["sparse"], points=points_sparse)
client.upsert(collection_name=collections["zero"], points=points_zero)
client.upsert(collection_name=collections["uniform"], points=points_uniform)
client.upsert(collection_name=collections["normal"], points=points_normal)

print("Data upserted into Qdrant Cloud collections successfully!")


Data upserted into Qdrant Cloud collections successfully!


In [117]:
import requests
GROQ_API_KEY="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"
GROQ_MODEL="llama-3.3-70b-versatile"

In [ ]:
import streamlit as st
import numpy as np
from qdrant_client import QdrantClient
from groq import Groq  # Import Groq API

# -------------------------------
# Load Configuration (Replace with actual values)
# -------------------------------
QDRANT_URL = "https://64c9b924-a747-4c21-874f-f6901dc0431e.us-east-1-0.aws.cloud.qdrant.io:6333"
QDRANT_API_KEY = "vgywMs0OrU6zw1BkgnJ6sQ8CAbp2XJ7spYIgQFoRaEjAqCAXLlp3Rg"
GROQ_API_KEY = "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"
GROQ_MODEL = "llama-3.3-70b-versatile"  # Change if using a different model



In [120]:
# Cache Qdrant Connection
# -------------------------------
@st.cache_resource
def get_qdrant_client():
    return QdrantClient(host=QDRANT_URL, api_key=QDRANT_API_KEY, https=True)

client = get_qdrant_client()

# -------------------------------
# Initialize Groq API Client
# -------------------------------
groq_client = Groq(api_key=GROQ_API_KEY)

# -------------------------------
# Streamlit Chat Interface Layout
# -------------------------------
st.title("Qdrant & Groq LLM Chatbot")
st.write("Select an embedding method, ask your query, and get responses from Llama-3.3.")

# Dropdown to select the embedding method (collection)
selected_method = st.selectbox("Select Embedding Method", list(collections.keys()))
collection_name = collections[selected_method]

# Text input for the query
query = st.text_input("Enter your query:", "What is the age of Vinod?")

# -------------------------------
# Function to Convert Query to Vector
# -------------------------------
def query_to_vector(query, method):
    n = 10  # Assume fixed embedding dimension
    if method == "sparse_vectors":
        vec = [0.0] * n
        if "vinod" in query.lower():
            vec[1] = 1.0  # Simulating sparse vector behavior
        else:
            vec[0] = 1.0
        return vec
    elif method == "zero_vectors":
        return [0.0] * n
    elif method == "uniform_vectors":
        return np.random.uniform(0, 1, n).round(2).tolist()
    elif method == "normal_vectors":
        return np.random.normal(0.5, 0.1, n).round(2).tolist()
    else:
        return [0.0] * n

# -------------------------------
# Function to Generate Response using Groq LLM
# -------------------------------
def get_llm_response(query, context):
    prompt = f"User Query: {query}\nContext: {context}\nAnswer:"
    
    response = groq_client.chat.completions.create(
        model=GROQ_MODEL,
        messages=[{"role": "user", "content": prompt}],
        max_tokens=200
    )

    return response.choices[0].message.content.strip() if response.choices else "No response generated."

# -------------------------------
# Handle Query Submission
# -------------------------------
if st.button("Submit Query"):
    if query:
        # Convert the query into an embedding
        query_vector = query_to_vector(query, collection_name)
        st.write("**Query Vector:**", query_vector)
        
        # Perform similarity search in Qdrant
        search_result = client.search(
            collection_name=collection_name,
            query_vector=query_vector,
            limit=1
        )
        
        # Retrieve and display the result
        if search_result:
            result = search_result[0]
            name = result.payload.get("Name", "Unknown")
            age = result.payload.get("Age", "Unknown")
            context = f"Name: {name}, Age: {age}"
            
            # Send the retrieved data to LLM
            llm_response = get_llm_response(query, context)
            st.success(f"**LLM Response:** {llm_response}")
        else:
            st.error("No matching result found.")
    else:
        st.warning("Please enter a query.")

2025-02-18 15:11:08.042 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-18 15:11:08.043 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-18 15:11:08.043 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-18 15:11:08.043 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-18 15:11:08.056 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


ValueError: `host` param is not expected to contain protocol (http:// or https://). Try to use `url` parameter instead.

In [118]:
import streamlit as st
from groq import Groq 
groq_client = Groq(api_key=GROQ_API_KEY)

# -------------------------------
# Streamlit Chat Interface Layout
# -------------------------------
st.title("Qdrant & Groq LLM Chatbot")
st.write("Select an embedding method, ask your query, and get responses from Llama-3.3.")

# Dropdown to select the embedding method (collection)
selected_method = st.selectbox("Select Embedding Method", list(collections.keys()))
collection_name = collections[selected_method]

# Text input for the query
query = st.text_input("Enter your query:", "What is the age of Vinod?")

# -------------------------------
# Function to Convert Query to Vector
# -------------------------------
def query_to_vector(query, method):
    n = 10  # Assume fixed embedding dimension
    if method == "sparse_vectors":
        vec = [0.0] * n
        if "vinod" in query.lower():
            vec[1] = 1.0  # Simulating sparse vector behavior
        else:
            vec[0] = 1.0
        return vec
    elif method == "zero_vectors":
        return [0.0] * n
    elif method == "uniform_vectors":
        return np.random.uniform(0, 1, n).round(2).tolist()
    elif method == "normal_vectors":
        return np.random.normal(0.5, 0.1, n).round(2).tolist()
    else:
        return [0.0] * n

# -------------------------------
# Function to Generate Response using Groq LLM
# -------------------------------
def get_llm_response(query, context):
    prompt = f"User Query: {query}\nContext: {context}\nAnswer:"
    
    response = groq_client.chat.completions.create(
        model=GROQ_MODEL,
        messages=[{"role": "user", "content": prompt}],
        max_tokens=200
    )

    return response.choices[0].message.content.strip() if response.choices else "No response generated."

# -------------------------------
# Handle Query Submission
# -------------------------------
if st.button("Submit Query"):
    if query:
        # Convert the query into an embedding
        query_vector = query_to_vector(query, collection_name)
        st.write("**Query Vector:**", query_vector)
        
        # Perform similarity search in Qdrant
        search_result = client.search(
            collection_name=collection_name,
            query_vector=query_vector,
            limit=1
        )
        
        # Retrieve and display the result
        if search_result:
            result = search_result[0]
            name = result.payload.get("Name", "Unknown")
            age = result.payload.get("Age", "Unknown")
            context = f"Name: {name}, Age: {age}"
            
            # Send the retrieved data to LLM
            llm_response = get_llm_response(query, context)
            st.success(f"**LLM Response:** {llm_response}")
        else:
            st.error("No matching result found.")
    else:
        st.warning("Please enter a query.")

2025-02-18 15:07:56.524 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-18 15:07:56.526 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-18 15:07:56.527 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-18 15:07:56.528 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-18 15:07:56.528 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-18 15:07:56.532 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-18 15:07:56.534 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-18 15:07:56.536 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar